In [1]:
import streamlit as st
import pandas as pd
import sqlalchemy as sq

In [2]:
engine = sq.create_engine('sqlite:///find_me.db')
cnx = engine.connect()

In [3]:
pd.read_sql('location',cnx)

,current_loc,current_date,future_loc,future_date
0,current city,2021-02-21,new city,2021-02-28
